In [1]:
import tensorflow as tf
print(tf.__version__)

import os
config=tf.ConfigProto()
config.gpu_options.allow_growth=True
config.allow_soft_placement=True

import matplotlib
matplotlib.use('Agg')

import matplotlib.pyplot as plt
import numpy as np
import time

#to make directories
import pathlib

import sys

from utils import *

import argparse

1.13.1


In [2]:
trx = np.load("data/gan_processed_data/lab_x_mini.npy")

In [15]:
no_filters=[1, 16, 32, 64, 128, 256]
z_lat_dim=100
lat_dim=128
img_size_x = 16
img_size_y = 16
batch_size = 20
num_channels = 1
num_classes = 2
en_1hot = 0
std_init=0.01
SEED=1
acti='xavier'
intl=tf.truncated_normal_initializer(stddev=std_init, seed=SEED)
interp_val = 0

In [20]:
def upsample_layer(ip_layer, method=0, scale_factor=2, dim_list=None):
    '''
    2D upsampling layer with default image scale factor of 2.
    ip_layer : input feature map layer
    method = 0 --> Bilinear Interpolation
             1 --> Nearest Neighbour
             2 --> Bicubic Interpolation
    scale_factor : factor by which we want to upsample current resolution
    '''
    if(dim_list!=None):
        prev_height = dim_list[1]
        prev_width = dim_list[2]
    else:
        prev_height = ip_layer.get_shape().as_list()[1]
        prev_width = ip_layer.get_shape().as_list()[2]

    new_height = int(round(prev_height * scale_factor))
    new_width = int(round(prev_width * scale_factor))

    op = tf.image.resize_images(images=ip_layer,size=[new_height,new_width],method=method)

In [21]:
 def conv2d_layer(ip_layer,    # The previous/input layer.
                 name,                 # Name of the conv layer
                 bias_init=0,          # Constant bias value for initialization
                 kernel_size=(3,3),    # Width and height of each filter.
                 strides=(1,1),        # stride value of the filter
                 num_filters=32,       # Number of output filters.
                 padding='SAME',       # Padding - SAME for zero padding - i/p and o/p of conv. have same dimensions
                 use_bias=True,        #to use bias or not
                 use_relu=True,        # Use relu as activation function
                 use_batch_norm=False,# use batch norm on layer before passing to activation function
                 use_conv_stride=False,  # Use 2x2 max-pooling - obtained by convolution with stride 2.
                 training_phase=True,   # Training Phase
                 scope_name=None,      # scope name for batch norm
                 acti_type='xavier',   # weight and bias variable initializer type
                 dilated_conv=False,   # dilated convolution enbale/disable
                 dilation_factor=1):   # dilation factor
        '''
        Standard 2D convolutional layer
        '''
        # Num. channels in prev. layer.
        prev_layer_no_filters = ip_layer.get_shape().as_list()[-1]

        weight_shape = [kernel_size[0], kernel_size[1], prev_layer_no_filters, num_filters]
        bias_shape = [num_filters]

        strides_augm = [1, strides[0], strides[1], 1]

        if(scope_name==None):
            scope_name=str(name)+'_bn'

        with tf.variable_scope(name):

            weights = self.get_weight_variable(weight_shape, name='W',acti_type=acti_type)
            if(use_bias==True):
                biases = self.get_bias_variable(bias_shape, name='b', init_bias_val=bias_init)

            if(dilated_conv==True):
                op_layer = tf.nn.atrous_conv2d(ip_layer, filters=weights, rate=dilation_factor, padding=padding, name=name)
            else:
                if(use_conv_stride==False):
                    op_layer = tf.nn.conv2d(ip_layer, filter=weights, strides=strides_augm, padding=padding)
                else:
                    op_layer = tf.nn.conv2d(input=ip_layer, filter=weights, strides=[1, 2, 2, 1], padding=padding)

            #Add bias
            if(use_bias==True):
                op_layer = tf.nn.bias_add(op_layer, biases)

            if(use_batch_norm==True):
                op_layer = self.batch_norm_layer(ip_layer=op_layer,name=scope_name,training=training_phase)
            if(use_relu==True):
                op_layer = tf.nn.relu(op_layer)

            # Add Tensorboard summaries
            #_add_summaries(op_layer, weights, biases)

        #return op_layer,weights,biases
        return op_layer

In [11]:
#Generator - FCN variables
gen_c1_weights = tf.get_variable(name="gen_c1_weights",shape=[z_lat_dim,z_hid_dim], initializer=intl)
gen_c1_biases = tf.get_variable(name="gen_c1_biases",shape=[z_hid_dim], initializer=tf.constant_initializer(value=0))

#Discriminator - FCN variables
fcn_c1_weights = tf.get_variable(name="fcn_c1_weights",shape=[hid_dim, latent_dim], initializer=intl)
fcn_c1_biases = tf.get_variable(name="fcn_c1_biases",shape=[latent_dim], initializer=tf.constant_initializer(value=0))
fcn_c2_weights = tf.get_variable(name="fcn_c2_weights",shape=[latent_dim, latent_dim], initializer=intl)
fcn_c2_biases = tf.get_variable(name="fcn_c2_biases",shape=[latent_dim], initializer=tf.constant_initializer(value=0))
fcn_c3_weights = tf.get_variable(name="fcn_c3_weights",shape=[latent_dim,1], initializer=intl)
fcn_c3_biases = tf.get_variable(name="fcn_c3_biases",shape=[1], initializer=tf.constant_initializer(value=0))

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
#L: this will likely need to be changed along with the others. Original value is 4*4 which is what we have for the deformation
hid_dim=int(img_size_x*img_size_y*no_filters[4]/(4*4))
latent_dim=lat_dim

#L: changing this to 4*4 instead of 32*32 based on our reduction in dimensionality factor
z_hid_dim=int(img_size_x*img_size_y*no_filters[4]/(4*4))
#L: based on note on the top of the Generator network, dim_x is supposed to be the dimensions of the downsampled image,
#4x4 in the case of JB's unet. Setting this to img size/4 for now. previously was imgsize/32
dim_x = int(img_size_x/4)

# Placeholders
# input to the network
z = tf.placeholder(tf.float32, shape=[batch_size, z_lat_dim], name='z')
x_l = tf.placeholder(tf.float32, shape=[batch_size, img_size_x, img_size_y, num_channels], name='x_l')
x = tf.placeholder(tf.float32, shape=[None, img_size_x, img_size_y, num_channels], name='x')
x_unl = tf.placeholder(tf.float32, shape=[None, img_size_x, img_size_y, num_channels], name='x_unl')
if(en_1hot==1):
    y_l = tf.placeholder(tf.float32, shape=[None, img_size_x, img_size_y, num_classes], name='y_l')
else:
    y_l = tf.placeholder(tf.int32, shape=[None, img_size_x, img_size_y], name='y_l')
select_mask = tf.placeholder(tf.bool, name='select_mask')
train_phase = tf.placeholder(tf.bool, name='train_phase')
if(en_1hot==0):
    y_l_onehot=tf.one_hot(y_l,depth=num_classes)
else:
    y_l_onehot=y_l

In [23]:
############################################
## Generator Network
############################################
# Dense layer + Reshape reshape to down sampled image dimensions
gen_fcn_c1 = tf.matmul(z, gen_c1_weights) + gen_c1_biases
gen_fcn_relu_c1 = tf.nn.relu(gen_fcn_c1)
gen_fcn_reshaped = tf.reshape(gen_fcn_relu_c1 ,[-1,dim_x,dim_x,no_filters[4]])
print(gen_fcn_reshaped.shape)# Level 5 - Upsampling layer + Conv. layer

fs_de=2
scale_val=2
gen_up5 = upsample_layer(ip_layer=gen_fcn_reshaped, method=interp_val, scale_factor=scale_val)
print(gen_up5.shape)
gen_c5 = conv2d_layer(ip_layer=gen_up5,name='gen_c5', kernel_size=(fs_de,fs_de),num_filters=no_filters[4],use_relu=True, use_batch_norm=True, training_phase=train_phase)
print(gen_c5.shape)

# Level 4
gen_up4 = upsample_layer(ip_layer=gen_c5, method=interp_val, scale_factor=scale_val)
gen_c4 = conv2d_layer(ip_layer=gen_up4,name='gen_c4', kernel_size=(fs_de,fs_de),num_filters=no_filters[3],use_relu=True, use_batch_norm=True, training_phase=train_phase)
print(gen_c4.shape)

# Level 3
gen_up3 = upsample_layer(ip_layer=gen_c4, method=interp_val, scale_factor=scale_val)
gen_c3 = conv2d_layer(ip_layer=gen_up3,name='gen_c3', kernel_size=(fs_de,fs_de),num_filters=no_filters[2],use_relu=True, use_batch_norm=True, training_phase=train_phase)
print(gen_c3.shape)

# Level 2
gen_up2 = upsample_layer(ip_layer=gen_c3, method=interp_val, scale_factor=scale_val)
gen_c2 = conv2d_layer(ip_layer=gen_up2,name='gen_c2', kernel_size=(fs_de,fs_de),num_filters=no_filters[1],use_relu=True, use_batch_norm=True, training_phase=train_phase)
print(gen_c2.shape)

# Level 1
gen_up1 = upsample_layer(ip_layer=gen_c2, method=interp_val, scale_factor=scale_val)
gen_c1 = conv2d_layer(ip_layer=gen_up1,name='gen_c1', kernel_size=(fs_de,fs_de),num_filters=no_filters[1],use_relu=False, use_batch_norm=False, training_phase=train_phase)
print(gen_c1.shape)

(20, 4, 4, 128)


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
        # Conv. ops on input image
        conv_1a = layers.conv2d_layer(ip_layer=x_l,name='conv_1a',num_filters=no_filters[1], use_relu=True, use_batch_norm=True, training_phase=train_phase)
        conv_1b = layers.conv2d_layer(ip_layer=conv_1a,name='conv_1b',num_filters=no_filters[1], use_relu=True, use_batch_norm=True, training_phase=train_phase)

        # Concatenate features obtained by conv. ops on image and on 'z'
        gen_cat=tf.concat((gen_c1,conv_1b),axis=3)

        # More Conv. ops on concatenated feature maps
        conv_1c = layers.conv2d_layer(ip_layer=gen_cat,name='conv_1c',num_filters=no_filters[1], use_relu=True, use_batch_norm=True, training_phase=train_phase)
        conv_1d = layers.conv2d_layer(ip_layer=conv_1c,name='conv_1d',num_filters=no_filters[1], use_relu=True, use_batch_norm=True, training_phase=train_phase)
        conv_1e = layers.conv2d_layer(ip_layer=conv_1d,name='conv_1e',num_filters=2, use_relu=False, use_batch_norm=False, training_phase=train_phase)

        flow_vec = conv_1e

        # apply flow vector on the input image to get non-affine transformed image
        y_trans=tf.contrib.image.dense_image_warp(image=x_l,flow=flow_vec,name='dense_image_warp')